In [8]:
from skimage import feature
import numpy as np
import cv2
import os
from sklearn.svm import LinearSVC
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import random
import pickle

In [19]:
class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        self.numPoints = numPoints
        self.radius = radius

    def describe(self, image, eps=1e-7):
        lbp = feature.local_binary_pattern(image, self.numPoints, self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, self.numPoints + 3), range=(0, self.numPoints + 2))
        # print(hist, len(hist), sum(hist), lbp.shape[0]*lbp.shape[1])
        hist = hist.astype("float")
        hist /= (hist.sum() + eps) # values range from 0 to 1
        return hist

In [20]:
images = []
for folder in ['fake', 'real']:
    for image in os.listdir(folder):
        images.append(os.path.join(folder, image))

train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

random.shuffle(train_images)
random.shuffle(val_images)

# counting classes
# from collections import Counter
# train_labels = [image.split('\\')[0] for image in train_images]
# val_labels = [image.split('\\')[0] for image in val_images]
# print(Counter(train_labels))
# print(Counter(val_labels))

desc_24_8 = LocalBinaryPatterns(24, 8)
desc_8_2 = LocalBinaryPatterns(8, 2)
print(desc_24_8)



In [21]:
# pre-processing and then saving it. no need to generate again
data = []
labels = []
for image_path in tqdm(train_images):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist_24_8 = desc_24_8.describe(gray)
    hist_8_2 = desc_8_2.describe(gray)
    #print(hist_24_8)
    #print(hist_24_8.shape, hist_8_2.shape)
    hist = np.concatenate((hist_24_8, hist_8_2), axis =0)
    #print(hist.shape)

    labels.append(image_path.split('\\')[0])
    data.append(hist)

with open("X_train_24,8.txt", "wb") as fp:
    pickle.dump(data, fp)

with open("Y_train_24,8.txt", "wb") as fp:
    pickle.dump(labels, fp)

data = []
labels = []
for image_path in tqdm(val_images):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hist_24_8 = desc_24_8.describe(gray)
    hist_8_2 = desc_8_2.describe(gray)
    hist = np.concatenate((hist_24_8, hist_8_2), axis =0)
    #hist = desc.describe(gray)

    labels.append(image_path.split('\\')[0])
    data.append(hist)

with open("X_test_24,9.txt", "wb") as fp:
    pickle.dump(data, fp)

with open("Y_test_24,8.txt", "wb") as fp:
    pickle.dump(labels, fp)



100%|████████████████████████████████████████████████████████████████████████████████| 864/864 [00:40<00:00, 21.11it/s]


In [22]:
# training from saved X and Y
with open("X_train_24,8.txt", "rb") as fp:   # Unpickling
    X_train = pickle.load(fp)

with open("Y_train_24,8.txt", "rb") as fp:   # Unpickling
    Y_train = pickle.load(fp)

model = LinearSVC(C=100.0, random_state=42, max_iter=1000000, verbose=1)
print("Training")
model.fit(X_train, Y_train)

print(model.coef_)
print(model.intercept_)

filename = 'model_LBP_SVM.sav'
pickle.dump(model, open(filename, 'wb'))
del model

# loading model
model = pickle.load(open(filename, 'rb'))
print('loaded')


Training
[LibLinear][[-24.00773064  -7.34606109  10.93151293   5.83681635   4.22636129
    4.07901247  -3.90070668 -18.62161891 -32.79584195 -40.96164582
  -23.98390199   6.80349955  -3.41834494  37.2247956   60.66548792
   19.12590022   8.03534513   4.48271138   8.42317568  -2.33473009
  -15.94707336 -42.1043495  -22.91749441  22.11950954  20.7283618
   18.32209793  -0.23920615  -8.14706739  17.58328141   6.22604474
   15.84374534 -22.31147445 -36.56967638  10.97900247  12.83076659
   -3.53032776]]
[-7.3349116]
loaded
